In [2]:
# !pip install dowhy gcastle

In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import statsmodels.api as sm
from dowhy import gcm
from dowhy import CausalModel

In [4]:
# Sample Data
S = np.random.normal(loc=0.5, scale=1, size=2000)
X = 2*S + np.random.normal(loc=0, scale=0.5, size=2000)
Y = 3*S + 1.5*X + np.random.normal(loc=1,scale=0.5, size=2000)

# Interventional calculation

In [5]:
# Estimate causal effect using back-door criterion
# S -> X -> Y
#  \-------^

data = pd.DataFrame({'X': X, 'S': S})
data = sm.add_constant(data, prepend=True)
model = sm.OLS(Y, data) # Ordinary Linear Regression
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 1.446e+05
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        20:45:21   Log-Likelihood:                -1467.0
No. Observations:                2000   AIC:                             2940.
Df Residuals:                    1997   BIC:                             2957.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9818      0.013     77.041      0.000       0.957       1.007
X              1.4924      0.023     65.596      0.000       1.448       1.537
S              3.0238      0.047     64.408      0.000       2.932       3.116
==============================================================================
Omnibus:                        4.548   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.103   Jarque-Bera (JB):                4.512
Skew:                          -0.116   Prob(JB):                        0.105
Kurtosis:                       3.018   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
# Calculate P(Y|do(X=1))

X1 = np.array([1.0]*len(X))
Y1 = results.params['X']*X1 + results.params['S']*S + results.params['const']
np.average(Y1)


In [28]:
# Calculating Intervention using DoWhy GCM Model
# https://www.pywhy.org/dowhy/v0.12/user_guide/causal_tasks/what_if/interventions.html\
# Hint: you can also use gcm.auto.assign_causal_mechanisms (see: https://www.pywhy.org/dowhy/v0.10.1/user_guide/modeling_gcm/draw_samples.html)

# PROBLEM 1 IMPLEMENT INTERVENTION USING DoWHY GCM MODEL
# [ YOUR CODE HERE ]
graph = nx.DiGraph()
graph.add_edge('X', 'Y')
graph.add_edge('S', 'X')
graph.add_edge('S', 'Y')
data = pd.DataFrame({'X': X, 'S': S, 'Y': Y})
# nx.draw(graph, with_labels=True)

model = gcm.ProbabilisticCausalModel(graph)
gcm.auto.assign_causal_mechanisms(model, data)
gcm.fit(model, data)

samples = gcm.interventional_samples(model, {'X': lambda x: 1}, num_samples_to_draw=1000)
print(samples['Y'].mean())

Fitting causal mechanism of node S: 100%|██████████| 3/3 [00:00<00:00, 1332.94it/s]

4.088180829167435


# Counterfactual calculation

In [32]:
data_gcm2 = pd.DataFrame({'X': X, 'S': S, 'Y': Y})
causal_model2 = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('S', 'X'), ('S', 'Y')]))
gcm.auto.assign_causal_mechanisms(causal_model2, data_gcm2)
gcm.fit(causal_model2, data_gcm2)

gcm.counterfactual_samples(
    causal_model2,
    {'X': lambda x: 1},
    observed_data=pd.DataFrame(data=dict(X=[0], Y=[2], S=[0.5])))

Fitting causal mechanism of node S: 100%|██████████| 3/3 [00:00<00:00, 999.99it/s]


,S,X,Y
0,0.5,1,3.492418


# Causal Discovery

In [35]:
from castle.algorithms import PC, ICALiNGAM

2025-02-26 21:39:06,891 - /home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2025-02-26 21:39:06,900 - /home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [36]:
pc = PC()
pc_dataset = np.vstack([X, Y, S]).T
pc.learn(pc_dataset)

In [40]:
pc.causal_matrix

# PROBLEM 2 - WHY IS THE CAUSAL METRIX ALMOST COMPLETE?
# CAN YOU USE REFUTION TEST TO CHECK FOR CAUSALITY DIRECTION?
# IMPLEMENT AND DISCUSS

Tensor([[0, 1, 1],
        [1, 0, 1],
        [1, 1, 0]])

In [70]:
g = ICALiNGAM(max_iter=1000, thresh=0.7)

xys = [X, Y, S]
name = 'XYS'
def play(now, mk):
    if len(now) == len(xys)-1:
        a = []
        n = ""
        for i in range(len(xys)-1):
            a.append(xys[now[i]])
            n += name[now[i]]
        dataset = np.vstack(a).T
        # print(dataset.shape)
        g.learn(dataset)
        print(n)
        print(g.causal_matrix)
        print()
        return
    for i in range(len(xys)):
        if mk[i] == 1:
            continue
        mk[i] = 1
        play(now + [i], mk)
        mk[i] = 0
play([], [0, 0, 0])

XY
[[0 1]
 [0 0]]

XS
[[0 0]
 [0 0]]

YX
[[0 0]
 [0 0]]

YS
[[0 0]
 [0 0]]

SX
[[0 1]
 [0 0]]

SY
[[0 1]
 [0 0]]



/home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/andre/anaconda3/envs/causality/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


# Answer
**ANS:** PC's matrix almost full because X, Y, S are all have a causal relationship with each other in some way but PC can't determine the direction of the edge so it'll show all possible edges between them (markov equivalence class).

And by recuding the graph and test for all possible direction of each pair  of nodes, we can find the true causal relationship between them via ICA-Lingam algorithm (needed to increase threshold to overcome confusion between X and S).

In [37]:
N = 2000
a = np.random.uniform(0, 1, N)
b = np.random.uniform(3, 6, N)
c = a + b + .1 * np.random.uniform(-2, 0, N)
d = .7 * c + .1 * np.random.uniform(0, 1, N)
lingam_dataset = np.vstack([a, b, c, d]).T

In [38]:
lingam = ICALiNGAM(random_state=1)
lingam.learn(lingam_dataset)

In [39]:
lingam.weight_causal_matrix

Tensor([[0.   , 0.   , 1.005, 0.   ],
        [0.   , 0.   , 1.001, 0.   ],
        [0.   , 0.   , 0.   , 0.701],
        [0.   , 0.   , 0.   , 0.   ]])